In [ ]:
import os
import json

import xml.etree.ElementTree as ET
from pathlib import Path
from collections import defaultdict

### Sec 5.1.1 Parse Raw PCB Design file

#### PCB

In [ ]:
class PCBSummary:
    """
    Class to store and retrieve PCB summary information like dimensions and layer count.
    """
    def __init__(self, length=0, width=0, num_layers=0):
        """
        Initialize PCB summary with dimensions and layer count.
        
        Parameters:
        - length: The length of the PCB in mm
        - width: The width of the PCB in mm
        - num_layers: The number of layers in the PCB
        """
        self.length = length
        self.width = width
        self.num_layers = num_layers
    
    def to_dict(self):
        """
        Convert PCB summary to dictionary.
        
        Returns:
        - Dictionary with PCB summary information
        """
        return {
            "Size": {
                "Length": self.length, 
                "Width": self.width,},
            "Number_of_Layers": self.num_layers,
        }
    
def get_board_dimensions(brd_content):
    """
    Calculate the dimensions of the PCB by analyzing the board outline (wire elements in the plain section).
    
    Parameters:
    - brd_content: The root element of the parsed PCB design XML tree.
    
    Returns:
    - tuple: (length, width) representing the PCB dimensions in millimeters.
    """
    # Find all wire elements within the plain section (these define the board outline)
    plain_wires = brd_content.findall(".//board/plain/wire")
    
    # If no board outline is found, try to find other indicators of board size
    if not plain_wires:
        # Look for dimension layer wires
        plain_wires = brd_content.findall(".//board/plain/wire[@layer='20']")
        
        # If still no wires found, try rectangle elements
        if not plain_wires:
            rectangles = brd_content.findall(".//board/plain/rectangle")
            if rectangles:
                # Use the first rectangle as the board outline
                rectangle = rectangles[0]
                try:
                    x1 = float(rectangle.get("x1", 0))
                    y1 = float(rectangle.get("y1", 0))
                    x2 = float(rectangle.get("x2", 0))
                    y2 = float(rectangle.get("y2", 0))
                    return abs(x2 - x1), abs(y2 - y1)
                except (ValueError, TypeError):
                    # Handle case where coordinates aren't numbers
                    return 0, 0
    
    # If no board outline indicators are found
    if not plain_wires:
        return 0, 0
    
    # Extract coordinates from all wire elements
    plain_coordinates = []
    for wire in plain_wires:
        try:
            x1 = float(wire.get("x1", 0))
            y1 = float(wire.get("y1", 0))
            x2 = float(wire.get("x2", 0))
            y2 = float(wire.get("y2", 0))
            plain_coordinates.append((x1, y1, x2, y2))
        except (ValueError, TypeError):
            # Skip wires with invalid coordinates
            continue
    
    # If no valid coordinates were found
    if not plain_coordinates:
        return 0, 0
    
    # Find the minimum and maximum coordinates to determine the board dimensions
    min_x = min(min(coord[0], coord[2]) for coord in plain_coordinates)
    min_y = min(min(coord[1], coord[3]) for coord in plain_coordinates)
    max_x = max(max(coord[0], coord[2]) for coord in plain_coordinates)
    max_y = max(max(coord[1], coord[3]) for coord in plain_coordinates)
    
    # Calculate the board dimensions
    board_length = abs(max_x - min_x)
    board_width = abs(max_y - min_y)
    
    return board_length, board_width

def count_board_layers(brd_content):
    """
    Count the number of copper layers in the PCB design.
    
    Parameters:
    - brd_content: The root element of the parsed PCB design XML tree.
    
    Returns:
    - int: The number of copper layers (signal layers) in the PCB.
    """
    # Find all layer elements
    layers = brd_content.findall(".//layers/layer")
    
    # EAGLE PCB typically uses layer numbers 1-16 for copper layers
    # Layer 1 is Top, Layer 16 is Bottom for a 2-layer board
    # In multi-layer boards, intermediate layers are numbered 2-15
    
    start_counting = False
    num_layer = 0
    for layer in layers:
        name = layer.get("name", "")
        if name == "Top":
            start_counting = True
            num_layer += 1
        elif name == "Bottom":
            num_layer += 1
            break
        elif start_counting:
            num_layer += 1
    
    return max(num_layer, 1)  # Return at least 1 layer

#### Components

In [ ]:
class ComponentCounter:
    def __init__(self, root):
        """
        Initializes the component counter with dictionaries for various component types and stores the root of the XML tree.
        
        Parameters:
        - root: The root element of the parsed PCB design XML tree.
        """
        self.resistor = {}
        self.capacitor = {}
        self.inductor = {}
        self.transistor = {}
        self.OpAmp = {}
        self.processor = {}
        self.sensor = {} 
        self.crystal = {} 
        self.connector = {} 
        self.otherPart = {}
        self.root = root  # Store the root of the parsed XML tree for .brd file

        # Store component connectivity information
        self.component_signals = defaultdict(list)  # Maps components to signals they're on
        self.signal_components = defaultdict(list)  # Maps signals to components on them
        self.component_connections = defaultdict(set)  # Maps components to other components they connect to
        self.pin_counts = {}  # Tracks the number of pins for each component
        
        # Parse connectivity data
        self._parse_signals()

    def _parse_signals(self):
        """
        Parse signal information from the board to build connectivity data.
        This helps in understanding component correlation.
        """
        signals = self.root.findall(".//signals/signal")
        for signal in signals:
            signal_name = signal.attrib.get('name', '')
            
            # Get all component references in this signal
            contactrefs = signal.findall(".//contactref")
            components_in_signal = []
            
            for ref in contactrefs:
                component_name = ref.attrib.get('element', '')
                pad = ref.attrib.get('pad', '')
                
                # Skip empty references
                if not component_name:
                    continue
                
                components_in_signal.append(component_name)
                self.component_signals[component_name].append({
                    'signal': signal_name,
                    'pad': pad
                })
                
                # Track pin counts
                if component_name not in self.pin_counts:
                    self.pin_counts[component_name] = set()
                self.pin_counts[component_name].add(pad)
            
            # Add components to this signal
            self.signal_components[signal_name] = components_in_signal
            
            # Update connections between components
            for i, comp1 in enumerate(components_in_signal):
                for comp2 in components_in_signal[i+1:]:
                    if comp1 != comp2:
                        self.component_connections[comp1].add(comp2)
                        self.component_connections[comp2].add(comp1)

    def count_pins_for_element(self, component_name):
        """
        Counts the number of pins for a component based on connectivity data.
        
        Parameters:
        - component_name: The name of the component.
        
        Returns:
        The number of unique pins.
        """
        if component_name in self.pin_counts:
            return len(self.pin_counts[component_name])
        
        # Fallback to package-based pin counting if connectivity data doesn't have it
        return self._count_pins_from_package(component_name)

    def _count_pins_from_package(self, component_name):
        """
        Counts the number of SMD pins for a component by examining its package.
        
        Parameters:
        - component_name: The name of the component.
        
        Returns:
        The number of SMD pins.
        """
        element = self.root.find(f".//elements/element[@name='{component_name}']")
        if element is None:
            return 0
            
        library_name = element.attrib.get('library', '')
        package_name = element.attrib.get('package', '')
        
        # Find the specific library element
        library = self.root.find(f".//library[@name='{library_name}']")
        if library is not None:
            # Within the found library, locate the specific package
            package = library.find(f".//packages/package[@name='{package_name}']")
            if package is not None:
                # Count all SMD elements within this package
                smd_elements = package.findall(".//smd")
                return len(smd_elements)
        return 0
    
    def identify_component_type(self, component_name, library_name, value, package_name=""):
        """
        Identifies the component type based on name, library, value, and connectivity patterns.
        
        Parameters:
        - component_name: The name of the component.
        - library_name: The library the component belongs to.
        - value: The component value.
        - package_name: The package of the component, if available.
        
        Returns:
        A string representing the determined component type.
        """
        # First check based on component naming convention
        if component_name.startswith('R'):
            return 'resistor'
        elif component_name.startswith('C'):
            return 'capacitor'
        elif component_name.startswith('L'):
            return 'inductor'
        elif component_name.startswith('JP') or component_name.startswith('J') or library_name == 'pinhead':
            return 'connector'
        elif component_name.startswith('X') or 'crystal' in library_name.lower() or 'xtal' in library_name.lower():
            return 'crystal'
        elif component_name.startswith('T') or component_name.startswith('Q'):
            return 'transistor'
        
        # Check based on connectivity patterns
        pin_count = self.count_pins_for_element(component_name)
        
        # Check for I2C connections
        is_on_i2c = any(signal.get('signal') in ['SDA', 'SCL'] for signal in self.component_signals.get(component_name, []))
        
        # Transistors typically have 3 pins
        if pin_count == 3:
            return 'transistor'
        
        # Op-amps often have 8 pins
        if pin_count == 8 and not is_on_i2c:
            return 'OpAmp'
        
        # Processors typically have many pins and connections
        if pin_count >= 16:
            return 'processor'
        
        # I2C sensors typically have 8 pins and are on I2C bus
        if is_on_i2c and 4 <= pin_count <= 16:
            return 'sensor'
        
        # Default to otherPart if we can't determine
        return 'otherPart'
    
    def add_component(self, element):
        """
        Extracts component information, classifies it, and counts it.
        
        Parameters:
        - element: The XML Element representing a component.
        """
        name = element.attrib.get('name', '')
        library_name = element.attrib.get('library', '')
        package_name = element.attrib.get('package', '')
        value = element.attrib.get('value', '')
        
        # Determine component type using the improved identification method
        component_type = self.identify_component_type(name, library_name, value, package_name)
        
        # Basic components like resistors, capacitors, and inductors
        if component_type in ['resistor', 'capacitor', 'inductor']:
            category = getattr(self, component_type)
            package_size = ''.join(filter(str.isdigit, package_name))
            
            if package_size in category:
                category[package_size] += 1
            else:
                category[package_size] = 1
            return
        
        # For more complex components, store detailed information
        category = getattr(self, component_type)
        pin_count = self.count_pins_for_element(name)
        
        # Extract additional attributes
        description = ""
        company_info = ""
        
        for child in element:
            if 'name' in child.attrib:
                if child.attrib['name'] == 'DESCRIPTION':
                    description = child.attrib.get('value', '')
                elif child.attrib['name'] == 'PACKAGE':
                    package_info = child.attrib.get('value', '').split()
                    if package_info:
                        package_name = package_info[0]
                        company_info = ' '.join(package_info[1:])
        
        # Get connection information for correlation analysis
        connections = []
        if name in self.component_connections:
            connections = list(self.component_connections[name])
        
        # Signals this component is connected to
        signals = [signal_info['signal'] for signal_info in self.component_signals.get(name, [])]
        unique_signals = list(set(signals))
        
        # Create component details based on component type
        if component_type == 'processor':
            component_details = {
                "Name": library_name,
                "Component": name,
                "Package": package_name,
                "Package_Area": "",  # Set to empty
                "Die_Size": "",  # Set to empty
                "Memory_Size": "",  # Set to empty
                "Number_of_Pins": pin_count,
                "GPIO_Count": "",  # Set to empty
                "Connected_To": connections,
                "Signals": unique_signals,
                "Power_Consumption": "",  # Set to empty
                "Process_Node": "",  # Set to empty
                "Count": 1,
                "Company_Info": company_info,
                "Description": description
            }
        elif component_type in ['transistor', 'OpAmp', 'sensor', 'crystal']:
            component_details = {
                "Name": library_name,
                "Component": name,
                "Package": package_name,
                "Package_Area": "",  # Set to empty
                "Die_Size": "",  # Set to empty
                "Number_of_Pins": pin_count,
                "Connected_To": connections,
                "Signals": unique_signals,
                "Process_Node": "",  # Set to empty
                "Count": 1,
                "Company_Info": company_info,
                "Description": description
            }
        else:
            # Basic details for other components
            component_details = {
                "Name": library_name,
                "Component": name,
                "Package": package_name,
                "Number_of_Pins": pin_count,
                "Connected_To": connections,
                "Signals": unique_signals,
                "Count": 1,
                "Company_Info": company_info,
                "Description": description
            }
        
        # Store or update the component in the appropriate category
        if name in category:
            category[name]["Count"] += 1
        else:
            category[name] = component_details
            
    
    def analyze_component_correlation(self):
        """
        Analyzes component correlation based on connectivity patterns.
        
        Returns:
        Dictionary with correlation information.
        """
        correlation_data = {}
        
        # Analyze I2C connections
        i2c_components = []
        for component, signals in self.component_signals.items():
            if any(signal['signal'] in ['SDA', 'SCL'] for signal in signals):
                i2c_components.append(component)
        
        if i2c_components:
            correlation_data['I2C_Bus'] = i2c_components
        
        # Find processor and its direct connections
        processor_components = list(self.processor.keys())
        if processor_components:
            processor = processor_components[0]  # Assuming one processor per board for simplicity
            processor_connections = {}
            
            for connected_comp in self.component_connections.get(processor, []):
                # Figure out which signals connect this component to the processor
                common_signals = []
                for signal, components in self.signal_components.items():
                    if processor in components and connected_comp in components:
                        common_signals.append(signal)
                
                if common_signals:
                    processor_connections[connected_comp] = common_signals
            
            correlation_data['processor_connections'] = processor_connections
        
        # Identify power supply components
        power_components = []
        for signal, components in self.signal_components.items():
            if signal in ['VDD', 'VCC', '3V3', '5V']:
                power_components.extend(components)
        
        if power_components:
            correlation_data['Power_Components'] = list(set(power_components))
        
        return correlation_data
    
    def get_components(self):
        """
        Returns a dictionary with all categorized components.
        
        Returns:
        A dictionary with component categories.
        """
        # Aggregate categories into a single dictionary
        categories = {
            "Resistor": self.resistor,
            "Capacitor": self.capacitor,
            "Inductor": self.inductor,
            "Transistor": self.transistor,
            "OpAmp": self.OpAmp,
            "Processor": self.processor,
            "Sensor": self.sensor,
            "Crystal": self.crystal,
            "Connector": self.connector,
            "Other_Parts": self.otherPart
        }
        
        components_summary = {}
        for category_name, category in categories.items():
            components_list = []
            for key, value in category.items():
                # For basic components, just include package and count
                if category_name in ["Resistor", "Capacitor", "Inductor"]:
                    components_list.append({"Package": key, "Count": value})
                else:
                    # For more complex components, include all details
                    if isinstance(value, dict):
                        component = value.copy()
                        components_list.append(component)
            components_summary[category_name] = components_list
        
        # Add correlation analysis
        components_summary["Component_Correlation"] = self.analyze_component_correlation()
        
        return components_summary


def parse_brd_pcb(brd_file_path):
    """
    Parses a .brd PCB design file and returns component information.
    
    Parameters:
    - brd_file_path: Path to the .brd file.
    
    Returns:
    A dictionary with component information and correlation data.
    """
    # Parse the board file
    brd_tree = ET.parse(brd_file_path)
    brd_content = brd_tree.getroot()

    # Get board dimensions
    length, width = get_board_dimensions(brd_content)
    
    # Get number of layers
    num_layers = count_board_layers(brd_content)
    
    # Create PCB summary
    pcb_summary = PCBSummary(length, width, num_layers)
    
    # Get elements
    elements_info = brd_content.findall(".//elements/element")
    
    # Create component counter
    components_summary = ComponentCounter(brd_content)
    
    # Analyze each component
    for element in elements_info:
        components_summary.add_component(element)

    # Create the final result dictionary
    result = {"board": pcb_summary.to_dict()}
    result.update(components_summary.get_components())

    
    # Return the analyzed components
    return result

#### Test

In [ ]:
# Example usage:
# Specify the path to your .brd file
brd_file = 'toy_pcb_design_files/IoT_temp_sensor.brd'

# Analyze the PCB
components = parse_brd_pcb(brd_file)

# Print the results
print(json.dumps(components, indent=2))

In [ ]:
components["Sensor"]

### Sec 5.1.2 Call External API

#### Digikey API

In [ ]:
# pip install git+https://github.com/hurricaneJoef/digikey-api.git
import digikey
from digikey.v4.productinformation import KeywordRequest

# Determine the current directory of this Python script
script_directory = os.getcwd()

# Create a cache directory path under the script's directory
CACHE_DIR = str(Path(script_directory) / 'digikey_api_cache')
# print(CACHE_DIR)

"""
Initializes the Digikey API with client credentials and configures the cache directory.

Parameters:
- cache_dir: The directory to store cache data from Digikey API responses.
"""
os.environ['DIGIKEY_CLIENT_ID'] = 'YOUR CLIENT ID'  # Replace with your actual Client ID
os.environ['DIGIKEY_CLIENT_SECRET'] = 'YOUR CLIENT SECRET'  # Replace with your actual Client Secret
os.environ['DIGIKEY_CLIENT_SANDBOX'] = 'False'
os.environ['DIGIKEY_STORAGE_PATH'] = CACHE_DIR

# Ensure the cache directory exists
Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

# Update to include additional component categories
def call_digikey_update_basic(components_dict):
    """
        Updates the PCB component table with additional details fetched from the Digikey API.

        Parameters:
        - components_dict: The component table to update.
    """
    # Define the component categories to update
    categories = ["Transistor", "OpAmp", "Processor", "Sensor"]
    
    for category in categories:
        for component in components_dict[category]:
            # Search for parts using the component's name or another relevant attribute
            component_name = component["Name"]
            search_request = KeywordRequest(keywords=component_name, limit=1)
            result = digikey.keyword_search(body=search_request)

            if result.products:  # Check if any product is returned
                response_data = result.products[0].to_dict()

                # Update details based on search results, e.g., "Package_Area"
                # This part assumes that the API's response structure and how you update your component details are similar to what was previously shown
                # For Lin Shi
                for parameter in response_data.get('parameters', []):
                    if parameter.get('parameter') == 'Supplier Device Package':
                        component['Package_Area'] = parameter.get('value')
                    elif parameter.get('parameter') == 'Memory Size':
                        component['Memory_Size'] = parameter.get('value')
                    elif parameter.get('parameter') == 'GPIO':
                        component['GPIO_Count'] = parameter.get('value')

            else:
                print(f"No result found for: {component}")

# Call the function to update components["Processor"]
call_digikey_update_basic(components)

### Sec 5.2.1 Infer Die Area

In [ ]:
def estimate_die_area(package_area):
    """
    Estimates the die size of an IC based on its package area and type.
    
    Parameters:
    - package_area: A string representing the package type and area, e.g., "48-QFN (6x6)"
    
    Returns:
    The estimated die size in the same units squared as provided in package_area.
    """

    # Map package types to coefficients for estimating die size
    # For Lin Shi
    die_size_coefficients = {
        'BGA': 0.8,  # This is a placeholder value since there are different values for large and small BGA
        'LGA': 0.8,  # Added manually, assume QFN is the same as BGA
        'DIP': 0.5,
        'PLCC': 0.5,
        'QFP': 0.5,
        'QFN': 0.5, # Added manually, assume QFN is the same as QFP
        'SO': 0.6,
        'SSOP': 0.5,
        'TQFP': 0.5,
        'TSOP': 0.6,
        'TSSOP': 0.6,
        'WLP': 0.9,
        'WLCSP': 0.9 # Added manually, assume WLCSP is the same as WLP
    }
    
    # Extract the package type from the string
    package_type = ''.join(filter(str.isalpha, package_area.split(' ')[-2]))
    
    # Extract the dimensions from the string
    dimensions = package_area.split(' ')[-1].strip('()').split('x')
    
    # If multiple dimensions are found, use them to calculate the die size
    if len(dimensions) == 2:
        width, length = float(dimensions[0]), float(dimensions[1])
        area = width * length
    else:  # If only one dimension is found, assume it's a square package
        side = float(dimensions[0])
        area = side * side
    
    # Look up the relevant ratio for the package type
    ratio = die_size_coefficients.get(package_type, 1)
    
    # Calculate the die size
    die_area = area * ratio * ratio
    
    return die_area

def update_die_size(components_dict):
    """
    Updates the die size for each IC component in the components_dict based on its package area.
    
    Parameters:
    - components_dict: The PCB component table object containing IC details.
    """
    
    # Define the component categories to update
    categories = ["Transistor", "OpAmp", "Processor", "Sensor"]
    
    for category in categories:
        for component in components_dict[category]:
        # for IC, attributes in components.items():
            package_area = component['Package_Area']
            if package_area:
                component['Die_Size'] = estimate_die_area(package_area)

# # Sanity Test
# package_area = '48-QFN (6x6)'
# die_size = estimate_die_area(package_area)
# print(die_size)

update_die_size(components)

### Save to JSON for Comparison

In [ ]:
# Save the components and their counts to a JSON file
with open('example.json', 'w', encoding='utf-8') as file:
    json.dump(components, file, ensure_ascii=False, indent=4)